Import libraries

In [1]:
from dataset.dataset import load_data
from models import MRnet
from config import config
import torch
from torch.utils.tensorboard import SummaryWriter
from utils.utils import _train_model, _evaluate_model, _get_lr
import time
import torch.utils.data as data
import os

Method for training a model

In [2]:
"""Performs training of a specified model.
    
Input params:
    config_file: Takes in configurations to train with 
"""

def train(config : dict):
    """
    Function where actual training takes place

    Args:
        config (dict) : Configuration to train with
    """
    
    print('Starting to Train Model...')

    train_loader, val_loader, train_wts, val_wts = load_data(config['task'])

    print('Initializing Model...')
    model = MRnet()
    if torch.cuda.is_available():
        model = model.cuda()
        train_wts = train_wts.cuda()
        val_wts = val_wts.cuda()

    print('Initializing Loss Method...')
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=train_wts)
    val_criterion = torch.nn.BCEWithLogitsLoss(pos_weight=val_wts)

    if torch.cuda.is_available():
        criterion = criterion.cuda()
        val_criterion = val_criterion.cuda()

    print('Setup the Optimizer')
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=3, factor=.3, threshold=1e-4, verbose=True)
    
    starting_epoch = config['starting_epoch']
    num_epochs = config['max_epoch']
    patience = config['patience']
    log_train = config['log_train']
    log_val = config['log_val']

    best_val_loss = float('inf')
    best_val_auc = float(0)

    print('Starting Training')

    writer = SummaryWriter(comment='lr={} task={}'.format(config['lr'], config['task']))
    t_start_training = time.time()

    for epoch in range(starting_epoch, num_epochs):

        current_lr = _get_lr(optimizer)
        epoch_start_time = time.time()  # timer for entire epoch

        print('Started Training')
        train_loss, train_auc = _train_model(
            model, train_loader, epoch, num_epochs, optimizer, criterion, writer, current_lr, log_every=log_train)

        print('train loop ended, now val')
        val_loss, val_auc = _evaluate_model(
            model, val_loader, val_criterion,  epoch, num_epochs, writer, current_lr, log_val)

        writer.add_scalar('Train/Avg Loss', train_loss, epoch)
        writer.add_scalar('Val/Avg Loss', val_loss, epoch)

        scheduler.step(val_loss)

        t_end = time.time()
        delta = t_end - epoch_start_time

        print("train loss : {0} | train auc {1} | val loss {2} | val auc {3} | elapsed time {4} s".format(
            train_loss, train_auc, val_loss, val_auc, delta))

        print('-' * 30)

        writer.flush()

        if val_auc > best_val_auc:
            best_val_auc = val_auc

        if bool(config['save_model']):
            file_name = 'model_{}_{}_val_auc_{:0.4f}_train_auc_{:0.4f}_epoch_{}.pth'.format(config['exp_name'], config['task'], val_auc, train_auc, epoch+1)
            torch.save({
                'model_state_dict': model.state_dict()
            }, './weights/{}/{}'.format(config['task'],file_name))

    t_end_training = time.time()
    print(f'training took {t_end_training - t_start_training} s')
    writer.flush()
    writer.close()

Train the model

In [3]:
print('Training Configuration')
print(config)

train(config=config)

print('Training Ended...')

Training Configuration
{'max_epoch': 50, 'log_train': 50, 'lr': 1e-05, 'starting_epoch': 0, 'batch_size': 1, 'log_val': 10, 'task': 'combined', 'weight_decay': 0.01, 'patience': 5, 'save_model': 1, 'exp_name': 'test'}
Starting to Train Model...
Loading Train Dataset of combined task...
Weights for loss is :  tensor([0.2377, 4.4327, 1.8463])
Loading Validation Dataset of combined task...
Weights for loss is :  tensor([0.2632, 1.2222, 1.3077])
Initializing Model...


/home/unc6kr/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/unc6kr/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Initializing Loss Method...
Setup the Optimizer
Starting Training
Started Training


/home/unc6kr/.local/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[Epoch: 1 / 50 | Batch : 50 / 1130 ]| Avg Train Loss 0.9111 | Train Avg AUC : 0.4751 abnorm:0.3995 acl:0.5591 meni:0.4667 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 100 / 1130 ]| Avg Train Loss 0.8623 | Train Avg AUC : 0.4737 abnorm:0.4356 acl:0.4786 meni:0.5068 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 150 / 1130 ]| Avg Train Loss 0.8197 | Train Avg AUC : 0.4837 abnorm:0.4115 acl:0.4907 meni:0.5489 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 200 / 1130 ]| Avg Train Loss 0.8014 | Train Avg AUC : 0.5071 abnorm:0.4349 acl:0.5271 meni:0.5593 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 250 / 1130 ]| Avg Train Loss 0.786 | Train Avg AUC : 0.5431 abnorm:0.4833 acl:0.5438 meni:0.6022 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 300 / 1130 ]| Avg Train Loss 0.7822 | Train Avg AUC : 0.5575 abnorm:0.4639 acl:0.5803 meni:0.6284 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 350 / 1130 ]| Avg Train Loss 0.7769 | Train Avg AUC : 0.5642 abnorm:0.4807 acl:0.5745 meni:0.6374 | lr : 1e-05
[Epoch: 1 / 50 | Batch : 400 / 1130 ]| Avg Train L